In [2]:
%pip install -U accelerate
%pip install transformers
%pip install transformers[torch]
%pip install spacy
%pip install unidecode
%pip install datasets
%pip install wandb
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import spacy
import re
import pandas as pd
from unidecode import unidecode
import wandb
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset, concatenate_datasets

In [4]:
data = pd.read_csv('../data/train.csv')
texts = data['text'].to_list()
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
texts[:30]

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask. Canada',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA 

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
def clean_text(texts):
    clean_texts = []
    for text in nlp.pipe(texts):
    #     clean_texts.append(" ".join([token.text.strip("#").lower() for token in text if not token.is_punct]))
        sen = []
        for t in text:
            if t.is_punct:
                continue
            t = '@user' if t.text.startswith('@') and len(t) > 1 else t.text
            t = 'http' if t.startswith('http') else t
            sen.append(t.strip('#').lower())
        clean_texts.append(" ".join(sen))
        #     print(sen)
        #     print([(token,token.is_punct) for token in text])
        #     clean_texts = [unidecode(re.sub(r"http\S+", "", text).strip()) for text in clean_texts]
    
    return clean_texts

data["final_text"] = clean_text(texts)

In [8]:
data

,id,keyword,location,text,target,final_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive wildfires evacuation ord..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding a bridge collapse int...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,@user @user the out of control wild fires in c...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m1.94 01:04 utc]?5 km s of volcano hawaii http
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigating after an e bike collided ...


In [9]:
# Loading the Hugging Face pretrained sequence classification model
model_name='cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [10]:
df = pd.DataFrame()
df['labels'] = data['target']
df['text'] = data['final_text']

In [11]:
df_train, df_val = train_test_split(df, test_size=0.2)

In [12]:
train_ds = Dataset.from_pandas(df_train, split='train')
val_ds = Dataset.from_pandas(df_val, split='test')

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
def preprocess_function(inp):
    return tokenizer(inp['text'], truncation=True)

In [15]:
train_ds_tk = train_ds.map(preprocess_function, batched=True)
val_ds_tk = val_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [16]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    label = pred.label_ids
    preds = pred.predictions.argmax(-1)
    prec, rec, f1, _ = precision_recall_fscore_support(label, preds, average='binary')
    acc = accuracy_score(label, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': prec,
        'recall': rec
    }

In [17]:
# Model training on Train dataset and performance on Validation dataset
training_args = TrainingArguments(
    output_dir="./results",         # set output directory
    learning_rate=1e-5,             # we can customize learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="wandb",              # enable logging to W&B
    logging_strategy='steps',
    logging_steps=100,              # we will log every 100 steps
    evaluation_strategy='steps',    # check evaluation metrics on a given # of steps
    eval_steps=100,                 # we will perform evaluation every 100 steps
    eval_accumulation_steps=1,      # report evaluation results after each step
    metric_for_best_model='f1',
    load_best_model_at_end=False
)

# automatically log model to W&B at the end
%env WANDB_LOG_MODEL=true

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tk,
    eval_dataset=val_ds_tk,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()

env: WANDB_LOG_MODEL=true


/Users/tarunbirgambhir/miniforge3/envs/tensorflow/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: tarun0131. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/3810 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# Reinitializing model for final training
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)

# Combine the training and validation datasets
train_ds_complete_tk = concatenate_datasets([train_ds_tk, val_ds_tk])

# Model training on entire dataset
training_args = TrainingArguments(
    output_dir="./results", 
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,   
    weight_decay=0.01,
    report_to="wandb",
    logging_strategy='steps',
    logging_steps=100,
)

%env WANDB_LOG_MODEL=true

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_complete_tk,  # Use the full dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


env: WANDB_LOG_MODEL=true


/Users/tarunbirgambhir/miniforge3/envs/tensorflow/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/476 [00:00<?, ?it/s]

{'loss': 0.5182, 'learning_rate': 7.899159663865547e-06, 'epoch': 0.21}
{'loss': 0.3894, 'learning_rate': 5.798319327731093e-06, 'epoch': 0.42}
{'loss': 0.3983, 'learning_rate': 3.6974789915966393e-06, 'epoch': 0.63}
{'loss': 0.4204, 'learning_rate': 1.5966386554621848e-06, 'epoch': 0.84}
{'train_runtime': 332.7459, 'train_samples_per_second': 22.879, 'train_steps_per_second': 1.431, 'train_loss': 0.4283043116080661, 'epoch': 1.0}


train/epoch,▁▃▅▇█
train/global_step,▁▃▅▇█
train/learning_rate,█▆▃▁
train/loss,█▁▁▃
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,476


In [ ]:
# pipeline to create the predictions file
from transformers import TextClassificationPipeline

inference_on_test_pipe = TextClassificationPipeline(model=model.to('cpu'), tokenizer=tokenizer)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
test_df = pd.read_csv('../data/test.csv')
test_clean_text = clean_text(test_df['text'].to_list())
scores = inference_on_test_pipe(test_clean_text)

In [ ]:
processed_scores = [int(s['label'].split('_')[1]) for s in scores]
test_df['target'] = processed_scores
df = test_df[['id', 'target']]
df.to_csv('../submissions.csv',index=False)